In [1]:
import torch
import warnings
import torch.nn as nn

warnings.filterwarnings(action='ignore')

c:\Users\korea\anaconda3\envs\py39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
!pip install torchsummary
from torchsummary import summary

#### Layer종류 & Block화

In [3]:
# linear layer
x = torch.randn(32,10) # Batch,Features
linear_layer = nn.Linear(in_features=10, out_features=1)
print(linear_layer(x).shape)

torch.Size([32, 1])


In [4]:
# convolution layer
imgs = torch.randn(32,3,64,64) #B, C, H, W -> B, Featuremap, H//2,w//2
conv_layer = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=(2,2), stride=(2,2))
print(conv_layer(imgs).shape)

torch.Size([32, 16, 32, 32])


In [5]:
# activation functions
x = torch.FloatTensor([[-1,1]])
relu = nn.ReLU(inplace=True)
sigmoid = nn.Sigmoid()
tanh = nn.Tanh()
print(x.shape, x, relu(x))

torch.Size([1, 2]) tensor([[-1.,  1.]]) tensor([[0., 1.]])


In [6]:
# 1D-pooling layer
x = torch.FloatTensor([[-1,1,2,6]])
maxpool = nn.MaxPool1d(kernel_size=2,stride=2)
avgpool = nn.AvgPool1d(kernel_size=2,stride=2)
print(x, maxpool(x), avgpool(x))

tensor([[-1.,  1.,  2.,  6.]]) tensor([[1., 6.]]) tensor([[0., 4.]])


In [7]:
# 2D-pooling layer
imgs = torch.randn(32,3,64,64)
maxpool = nn.MaxPool2d(kernel_size=(2,2),stride=(2,2))
avgpool = nn.AvgPool2d(kernel_size=(2,2),stride=(2,2))
print(maxpool(imgs).shape, )

torch.Size([32, 3, 32, 32])


In [8]:
# batchnorm
x = torch.FloatTensor([[1,2,3],
                       [2,2,2],
                       [3,2,1]])
batchnorm = nn.BatchNorm1d(num_features=3)
print(x, '\n', batchnorm(x))

tensor([[1., 2., 3.],
        [2., 2., 2.],
        [3., 2., 1.]]) 
 tensor([[-1.2247,  0.0000,  1.2247],
        [ 0.0000,  0.0000,  0.0000],
        [ 1.2247,  0.0000, -1.2247]], grad_fn=<NativeBatchNormBackward0>)


In [9]:
# dropout
x = torch.FloatTensor([[1,2,3],
                       [2,2,2],
                       [3,2,1]])
dropout = nn.Dropout(p=0.5)
dropout.train()
print(x, '\n', dropout(x))

tensor([[1., 2., 3.],
        [2., 2., 2.],
        [3., 2., 1.]]) 
 tensor([[2., 4., 6.],
        [0., 0., 0.],
        [0., 0., 0.]])


In [10]:
x = torch.randn(64,3,64,64)

conv_block = nn.Sequential(nn.Conv2d(in_channels=3,out_channels=16,kernel_size=(3,3),stride=(1,1),padding=(1,1)),
                           nn.BatchNorm2d(num_features=16),
                           nn.ReLU(inplace=True),
                           # nn.Dropout(p=0.1),
                           nn.MaxPool2d(kernel_size=(3,3),stride=(2,2), padding=(1,1))
)

summary(model=conv_block, input_data=(3,64,64), device='cpu')
print(conv_block(x).shape)

Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [-1, 16, 64, 64]          448
├─BatchNorm2d: 1-2                       [-1, 16, 64, 64]          32
├─ReLU: 1-3                              [-1, 16, 64, 64]          --
├─MaxPool2d: 1-4                         [-1, 16, 32, 32]          --
Total params: 480
Trainable params: 480
Non-trainable params: 0
Total mult-adds (M): 1.77
Input size (MB): 0.05
Forward/backward pass size (MB): 1.00
Params size (MB): 0.00
Estimated Total Size (MB): 1.05
torch.Size([64, 16, 32, 32])


#### Custom model 구축

In [11]:
"""
class CustomModel(torch.nn.Module):
  def __init__(self):
  원하는 nueral networks 모델 구조를 layer 별로 혹은 block화 하여 정의

  def forward(self, x):
  __init__에서 정의한 nueral networks에 모델을 입력하여 예측값을 얻는 forward 과정
"""

'\nclass CustomModel(torch.nn.Module):\n  def __init__(self):\n  원하는 nueral networks 모델 구조를 layer 별로 혹은 block화 하여 정의\n\n  def forward(self, x):\n  __init__에서 정의한 nueral networks에 모델을 입력하여 예측값을 얻는 forward 과정\n'

In [2]:
class MyLeNet(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()

        self.block1 = nn.Sequential(nn.Conv2d(in_channels=3,out_channels=6,kernel_size=(5,5),stride=(1,1),padding=(0,0)), # C1 layer
                           # nn.BatchNorm2d(num_features=6),
                           nn.Tanh(),
                           # nn.Dropout(p=0.1),
                           nn.AvgPool2d(kernel_size=(2,2),stride=(2,2), padding=(0,0))) # S2 layer
        
        self.block2 = nn.Sequential(nn.Conv2d(in_channels=6,out_channels=16,kernel_size=(5,5),stride=(1,1),padding=(0,0)), # C3 layer
                           # nn.BatchNorm2d(num_features=16),
                           nn.Tanh(),
                           # nn.Dropout(p=0.1),
                           nn.AvgPool2d(kernel_size=(2,2),stride=(2,2), padding=(0,0))) # S4 layer
        
        self.block3 = nn.Sequential(nn.Conv2d(in_channels=16,out_channels=120,kernel_size=(5,5),stride=(1,1),padding=(0,0)), # C5 layer
                           nn.Tanh())
        
        self.fc1 = nn.Sequential(nn.Linear(120,84), nn.Tanh())
        self.fc2 = nn.Linear(84,num_classes)
        
        self.flatten = nn.Flatten(start_dim=1, end_dim=-1)
        self.init_weights()
        
    def forward(self, x):
        # x : B, 3, 32, 32
        out = self.block1(x) # B, 6, 14, 14
        out = self.block2(out) # B, 16, 5, 5
        out = self.block3(out) # B, 120, 1, 1
        out = self.flatten(out) # B, 120*1*1
        out = self.fc1(out) # B, 84
        out = self.fc2(out) # B, num_classes
        return out
    
    # 모델의 파라미터를 초기화
    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight,
                                        mode='fan_out',
                                        nonlinearity='leaky_relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1.0)
                nn.init.constant_(m.bias, 0.0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight)
                nn.init.constant_(m.bias, 0.0)

In [3]:
summary(MyLeNet(num_classes=10), input_data=(3,32,32), device='cpu')

NameError: name 'summary' is not defined

In [4]:
x = torch.randn(size=(64,3,32,32))
lenet = MyLeNet(num_classes=2)

print(x.shape, lenet(x).shape)

torch.Size([64, 3, 32, 32]) torch.Size([64, 2])


#### Save & Load Model Parameters

In [5]:
# save model parameters
save_path = 'content/LGES2023_CNN/model.pt'
save_dict = {'model_A':lenet.state_dict(),
             #'model_B':lenet.state_dict(),
             #'optimizer':optimizer.load_state_dict(),
             }

torch.save(save_dict, save_path)

FileNotFoundError: [Errno 2] No such file or directory: 'content/LGES2023_CNN/model.pt'

In [16]:
# load model parameters
model = MyLeNet()
load_dict = torch.load(save_path, map_location='cpu')
parameters = load_dict['model_A']
model.load_state_dict(parameters,strict=False)

<All keys matched successfully>